# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
         
        for line in csvreader:
            full_data_rows_list.append(line) 
            

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS project2 WITH REPLICATION={ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('project2')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
query = "CREATE TABLE IF NOT EXISTS song_history_by_session_and_iteminSession"
query = query + "(SESSIONID int, ITEMINSESSION int, ARTIST text, SONG text, LENGTH float, primary key((SESSIONID),ITEMINSESSION))"
try:
    session.execute(query)
except Exception as e:
    print(e)      

In [9]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_history_by_session_and_iteminSession (SESSIONID, ITEMINSESSION, ARTIST, SONG, LENGTH)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT ARTIST,SONG,LENGTH from song_history_by_session_and_iteminSession where SESSIONID=338 and ITEMINSESSION=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
query = "CREATE TABLE IF NOT EXISTS song_history_by_user_and_session"
query = query + "( USERID int, SESSIONID int, ITEMINSESSION int, ARTIST text, USER text, SONG text, primary key((USERID,SESSIONID),ITEMINSESSION))"
try:
    session.execute(query)
except Exception as e:
    print(e)
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        user = line[1]+" "+line[4]
        query = "INSERT INTO song_history_by_user_and_session (USERID, SESSIONID, ITEMINSESSION, ARTIST, USER, SONG)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        session.execute(query, ( int(line[10]), int(line[8]), int(line[3]), line[0], user, line[9]))
        
                 

Run SELECT query to verify table model

In [12]:
query = "SELECT ARTIST,SONG,USER from song_history_by_user_and_session where USERID = 10 and SESSIONID = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row) 

Row(artist='Down To The Bone', song="Keep On Keepin' On", user='Sylvie Cruz')
Row(artist='Three Drives', song='Greece 2000', user='Sylvie Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', user='Sylvie Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user='Sylvie Cruz')


In [13]:
query = "create table if not exists user_by_song_heard "
query = "CREATE TABLE IF NOT EXISTS user_by_song_heard"
query = query + "( SONG text, USERID int, USER text, primary key((SONG), USERID))"
try:
    session.execute(query)
except Exception as e:
    print(e)
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        user = line[1]+" "+line[4]
        query = "INSERT INTO user_by_song_heard (SONG, USERID, USER)"
        query = query + " VALUES (%s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), user))
                   

Run SELECT query to verify table model

In [14]:
query = "SELECT SONG, USER from user_by_song_heard WHERE SONG ='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row) 

Row(song='All Hands Against His Own', user='Jacqueline Lynch')
Row(song='All Hands Against His Own', user='Tegan Levine')
Row(song='All Hands Against His Own', user='Sara Johnson')


### Drop the tables before closing out the sessions

In [15]:
query = "DROP TABLE song_history_by_session_and_iteminSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE song_history_by_user_and_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_by_song_heard"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [29]:
session.shutdown()
cluster.shutdown()